In [1]:
# PLACE ALL IMPORTS IN THIS CELL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# SET UP DATA

# convert csv to a pandas DataFrame format
train_df = pd.DataFrame.from_csv('data/train.csv')
test_df = pd.DataFrame.from_csv('data/test.csv')

/home/sn446/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.
/home/sn446/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """


In [3]:
# BASIC DATASET INFORMATION

# -- our train set is 1089 x 17
print(train_df.shape)
# -- our test set is 300 x 15 (the 2 missing columns are the label and a redundant field that resembles label)
print(test_df.shape)

(1089, 17)
(300, 15)


In [4]:
# FIGURE OUT LABEL DISTRIBUTION [~57% of dataset came from Android]

# create dataset with only positive labels -- 619 positive labels in training set
df_train_pos = train_df[train_df.label == 1]
# create dataset with only negative labels -- 470 negative labels in training set
df_train_neg = train_df[train_df.label == -1]
print("Number of positive [Android] training points: " + str(len(df_train_pos)))
print("Number of negative [iPhone] training points: " + str(len(df_train_neg)))
print("Percentage of training points from Android: " + str(len(df_train_pos)/train_df.shape[0]))

Number of positive [Android] training points: 619
Number of negative [iPhone] training points: 470
Percentage of training points from Android: 0.5684113865932048


In [8]:
# EXPLORE TIME FEATURE
print(train_df['created'])

7/12/16 0:56


# Features of Interest

In [ ]:
iphone = 0
http = 0
for i in range(1089):
    
    if "http" in train_df["text"][i]:
        http += 1
        if train_df["label"][i] == -1:
            iphone += 1
        
    
print(float(iphone)/http)

In [ ]:
android = 0
quote = 0

for i in range(1089):
    
    if '"' == train_df["text"][i][0]:
        quote += 1
        if train_df["label"][i] == 1:
            android += 1

print(float(android)/quote)
print(android)
print(quote)

# Vocabulary Creation

In [11]:
import keras.preprocessing.text as kt

In [35]:
words_frequency = {}

In [64]:
max_length = 0
for i in range(300):
    ws = kt.text_to_word_sequence(test_df['text'][i], filters = '"?!-')
    max_length = max(max_length, len(ws))
    for w in ws:
        if w in words_frequency.keys():
            words_frequency[w] += 1
        else:
            words_frequency[w] = 1

In [33]:
print(len(words_frequency.keys()))

4743


In [40]:
words = {}
words["http"] = 0

# Remove stuffs
for key in words_frequency.keys():
    if "http" in key:
        words["http"] += words_frequency[key]
    else:
        words[key] = words_frequency[key]

In [59]:
vocab_size = len(words.keys())
idx2word = {}
word2idx = {}
word2idx['END TOKEN'] = 0
idx2word[0] = 'END TOKEN'

counter = 0
for i in words.keys():
    counter += 1
    word2idx[i] = counter
    idx2word[counter] = i

4436

Created two dictionaries **word2idx** and **idx2word**

In [65]:
max_length = 32
hidden_size = 100

28

In [66]:
from keras.layers import Embedding, LSTM, Dropout, TimeDistributed, Activation, Input, Dense
from keras.models import Sequential

In [67]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, inputlength = max_length))
model.add(LSTM(hidden_size, return_sequences=False))
model.add(LSTM(hidden_size, return_sequences=False))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

NameError: name 'vocabulary' is not defined